In [9]:
import requests
import json
import csv

In [10]:
#this admin profile is created when the app is first launched (defined in run.py)
user = {'username': 'administrator', 'email': 'already_registered@123.comr','password': 'Admin10!', 'age': 26, 'activity_level': 2, 'gender_identity': 1}

## Login (user and admin)

In [11]:
r = requests.post('http://127.0.0.1:5000/login', json = user)
data = r.json()
try:
    access = data['access_token']
    refresh = data['refresh_token']
    print(data['message'])
    print('JSON Web Token: ' + access[:30])
except:
    print(data['message'])

Logged in as administrator. (admin)
JSON Web Token: eyJ0eXAiOiJKV1QiLCJhbGciOiJIUz


## Refresh Token

In [7]:
headers = { 'Authorization' : 'Bearer ' + refresh}

r = requests.post('http://127.0.0.1:5000/token/refresh', headers=headers)
data = r.json()
access = data['access_token']
print(data['message'])

Your access token was generated by using a refresh token.


# AdminResources

### User info (admin)

In [20]:
headers = { 'Authorization' : 'Bearer ' + access}
r = requests.post('http://127.0.0.1:5000/admin/user/info', headers=headers)
r.json()

### User preferences (admin)

In [24]:
headers = { 'Authorization' : 'Bearer ' + access}
r = requests.post('http://127.0.0.1:5000/admin/user/preferences', headers=headers)
r.json()

### Food data (dump) (admin)

In [26]:
headers = { 'Authorization' : 'Bearer ' + access}
r = requests.post('http://127.0.0.1:5000/admin/food/dump', headers=headers)
r.json()

### Food data (add) (admin)

In [16]:
food = {
'item_name' : 'juice box',
'latitude': 44, 
'longitude': -79, 
'restaurant_name' : 'Local bar',
'item_description' : 'This is secretly a salad.',
'nutrition' : json.dumps({'calories':100,'sugar':50,'total_fats': 120,'sat_fats':40,'cholesterol':150,'carbohydrates':100,'protein':20, 'fiber':30,'sodium':120,'magnesium':100,'potassium':40,'calcium':12}),
'price' : 2.30
}

In [27]:
headers = { 'Authorization' : 'Bearer ' + access}
r = requests.post('http://127.0.0.1:5000/admin/food/add', headers=headers, json = food)
r.json()

### Food data (edit) (admin)

In [70]:
headers = { 'Authorization' : 'Bearer ' + access}
r = requests.post('http://127.0.0.1:5000/admin/food/edit', headers=headers, json = food)
r.json()

### Food data (remove) (admin)

In [66]:
headers = { 'Authorization' : 'Bearer ' + access}
r = requests.post('http://127.0.0.1:5000/admin/food/remove', headers=headers, json = food)
r.json()

{'message': 'juice box at 44, -79 successfully deleted.'}

# UserResources

In [73]:
#test user 1 is joshuadrc, age 26, inactive, male.
user = {'username': 'joshuadrc', 'email': 'jsd42@duke.edu','password': 'Admin10!', 'age': 26, 'activity_level': 0, 'gender_identity': 1}

In [74]:
#test user 2 is juliedrc, age 42, active, female
user = {'username': 'juliedrc', 'email': 'joshuadrc@gmail.com','password': 'Admin10!', 'age': 42, 'activity_level': 2, 'gender_identity': 0}

## Registration (user)

In [ ]:
#create user if not already created
r = requests.post('http://127.0.0.1:5000/registration', json = user)
r.json()

## Login (user and admin)

In [ ]:
#login user
r = requests.post('http://127.0.0.1:5000/login', json = user)
data = r.json()
try:
    access = data['access_token']
    refresh = data['refresh_token']
    print(data['message'])
    print('JSON Web Token: ' + access[:30])
except:
    print(data['message'])

## Login Refresh (user and admin)

In [76]:
headers = { 'Authorization' : 'Bearer ' + refresh}

r = requests.post('http://127.0.0.1:5000/token/refresh', headers=headers)
data = r.json()
access = data['access_token']
print(data['message'])

Your access token was generated by using a refresh token.


## Preferences

### Add preference

In [25]:
headers = { 'Authorization' : 'Bearer ' + access}

data = {'preference': 'juice box', 
        'preference_action': 'add'}

r = requests.post('http://127.0.0.1:5000/editpreferences', json = data, headers = headers)
r.json()

### Remove preference

In [24]:
headers = { 'Authorization' : 'Bearer ' + access}

data = {'preference': 'kale salad', 
        'preference_action': 'remove'}

r = requests.post('http://127.0.0.1:5000/editpreferences', json = data, headers = headers)
r.json()

### List preferences

In [23]:
headers = { 'Authorization' : 'Bearer ' + access}
r = requests.get('http://127.0.0.1:5000/preferences', headers = headers)
r.json()['message']

## Recommendations

In [21]:
headers = { 'Authorization' : 'Bearer ' + access}
data = {'latitude': 44, 'longitude': -79, 'distance': 1000}

r = requests.post('http://127.0.0.1:5000/recommendation', headers = headers, json=data)

try:
    for i in json.loads(r.json()['data']):
        print('-----------------------------------------------------------')
        print('Based on: ' + user['username'] + '\'s preference for ' + i['based_on'])
        print('-----------------------------------------------------------')
        for item in i:
            print(f'{item}: {i[item]} \n')
except: 
    print(r.json())

-----------------------------------------------------------
Based on: administrator's preference for kale salad
-----------------------------------------------------------
item name: Broccoli Salad 

based_on: kale salad 

description: food description 

restaurant: Golden Corral 

price: 1.5 

location: [36.03952, -78.898941] 

distance: 885345.6663674267 

match to kale salad (previous item): 0.998454483286254 

nutrition score: 3 

recommended nutrition: {"calories": 600.0, "sugar": 54.0, "total_fats": 162.00000000000003, "sat_fats": 36.0, "cholesterol": 50.0, "carbohydrates": 330.00000000000006, "protein": 15.333333333333334, "fiber": 7.466666666666666, "sodium": 766.6666666666666, "magnesium": 106.66666666666667, "potassium": 1566.6666666666667, "calcium": 416.6666666666667} 

actual nutrition: {"calories": 200} 

embedding: [0.4157337546348572, 0.6945240795612335, 0.5400844663381577, 0.9694669842720032, 0.9319823086261749, 0.7396558225154877, 0.6211065649986267, 0.734076261520385

## Logout

In [22]:
headers = { 'Authorization' : 'Bearer ' + access}
r = requests.post('http://127.0.0.1:5000/logout/access', headers = headers)
r.json()

{'message': 'Access token has been revoked. You will be unable to use this token in the future.'}

# Bulk food add example

In [18]:
headers = { 'Authorization' : 'Bearer ' + access}
with open('Archive/food.csv', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        try:
            a = row[0].split('|')
            toadd = {
                'item_name' : str(a[0]),
                'latitude' : float(a[1]),
                'longitude' : float(a[2]),
                'restaurant_name' : a[3],
                'item_description' : 'food description',
                'price' : 1.50, 
                'nutrition': json.dumps({'calories': 200})
            }

            r = requests.post('http://127.0.0.1:5000/admin/food/add', headers = headers, json = toadd)
        except:
            continue